## 1. urllib 활용

In [27]:
import requests as r
import urllib.request as ur
import os
import pandas as pd
import time
import json
import socket

In [28]:
# 해당 디렉토리가 없으면 만드는 함수
def createFolder(dir_name):
    try:
        if not os.path.exists(dir_name):
            os.makedirs(dir_name)
    except OSError:
        print ('Error: Creating directory. ' +  dir_name)

In [36]:
key1 = "" # Your bing API key

In [37]:
#set your path
base_path = "" # Your base_path
os.chdir(base_path)

### 1. function to make image-files for one keyword

In [43]:
def make_imgs(item, num, apikey):
    # save start time to measure performance time
    start = time.time()
    # make a list to save error log
    error_log = []
    
    # set the directory
    os.chdir("./bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")
    
    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    
    # you can manage many query parameters. check out the documentation
    # https://docs.microsoft.com/en-us/bing/search-apis/bing-image-search/reference/query-parameters
    
    param = {"q" : item,
          "count" : num, #choose num of results from 1 to 150
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256, # set minimum height(pixels)
          "safeSearch" : "strict", # filter adult contents
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    
    # download images and labels using link result
    for i in range(0,len(test.get("value"))):
          file_name = item+"_"+str(i)
          print(file_name)
          print(test.get('value')[i].get('contentUrl'))
          json_file_name = f"{file_name}.json"
          file_path = os.getcwd()+'/'+json_file_name
          img_file_name = f"{file_name}.{test.get('value')[i].get('encodingFormat')}"
          #print(img_file_name)
          
          socket.setdefaulttimeout(3)
          
          try:
                ur.urlretrieve(test.get('value')[i].get('contentUrl'),img_file_name)
                data = {}
                data[file_name] = []
                data[file_name].append({
                      "contentUrl" : test.get('value')[i].get('contentUrl'),
                      "datePublished" : test.get('value')[i].get('datePublished'),
                      "encodingFormat" : test.get('value')[i].get('encodingFormat'),
                      "height" : test.get('value')[i].get('height'),
                      "hostPageUrl" : test.get('value')[i].get('hostPageUrl'),
                      "imageId" : test.get('value')[i].get('imageId'),
                      "width" : test.get('value')[i].get('width')
                })
                with open(file_path,'w') as outfile:
                      json.dump(data,outfile)
          except Exception as e:
                print(e)
                error_log.append(f"{img_file_name}: {e} ")

        
    # get back to the original path
    os.chdir("../..")
    time_taken = time.time()-start
    return time_taken, error_log

In [44]:
make_imgs("Joe Biden", 10, key1)

Joe Biden_0
https://dehayf5mhw1h7.cloudfront.net/wp-content/uploads/sites/1317/2020/08/04142449/GettyImages-1227830570.jpg
Joe Biden_1
https://newjerseyglobe.com/wp-content/uploads/2018/03/Joe-Biden-e1521136567698.jpg
HTTP Error 403: Forbidden
Joe Biden_2
https://api.time.com/wp-content/uploads/2019/09/joe-biden-ukraine-fundraising.jpg
Joe Biden_3
https://thingstoknow.com.ng/wp-content/uploads/2021/01/Joe-Biden-4-1024x682.jpg
Joe Biden_4
https://i1.wp.com/nypost.com/wp-content/uploads/sites/2/2020/06/joe-biden-basement.jpg?quality=90&strip=all&ssl=1
Joe Biden_5
https://upload.wikimedia.org/wikipedia/commons/0/0d/Joe_Biden,_official_photo_portrait_2-cropped.jpg
Joe Biden_6
https://s3.amazonaws.com/zweb-s3.uploads/ez2/wp-content/uploads/2020/08/GettyImages-1267430926_Fotor.jpg
Joe Biden_7
https://americarisingpac.org/wp-content/uploads/2019/06/Biden03-GettyImages-1149412192-e1560361192371.jpg
Joe Biden_8
https://www.welt.de/img/debatte/kommentare/mobile208189875/3552506897-ci102l-w1024/B

(10.504884243011475, ['Joe Biden_1.jpeg: HTTP Error 403: Forbidden '])

### 2. function to make image-file lists for several keywords

In [45]:
# 쿼리를 요청할 검색어 리스트 만들거나 불러오기
name_list = ['Barak Obama','Donald Trump','Joe Biden']

In [46]:
def make_img_list(lst, num, apikey):
  # 시작 시간 저장
  start = time.time()
  # 에러 로그 저장을 위한 리스트 만들기
  error_log = []
  # 주어진 리스트에 있는 모든 이름 검색
  for item in lst:

    # 폴더 만들고 경로 이동하기
    os.chdir(base_path+"/bing_img") # your designated directory
    createFolder(item)
    os.chdir(f"./{item}")

    url = "https://api.bing.microsoft.com/v7.0/images/search"
    headers = {
              "Ocp-Apim-Subscription-Key" : key1
              }
    param = {"q" : item,
          "count" : num,
          "imageType" : "Photo",
          "imageContent" : "Face",
          "minHeight" : 256,
          "safeSearch" : "strict",
          "color" : "ColorOnly"
          }
    s = r.Session()
    test = s.get(url,headers=headers, params=param).json()
    # 검색결과 링크에서 이미지 저장하기

    for i in range(0,len(test.get("value"))):
        file_name = item+"_"+str(i)
        print(file_name)
        print(test.get('value')[i].get('contentUrl'))
        json_file_name = f"{file_name}.json"
        file_path = os.getcwd()+'/'+json_file_name
        img_file_name = f"{file_name}.{test.get('value')[i].get('encodingFormat')}"
        #print(img_file_name)
        
        socket.setdefaulttimeout(3)

        
        try:
          ur.urlretrieve(test.get('value')[i].get('contentUrl'),img_file_name)
          data = {}
          data[file_name] = []
          data[file_name].append({
                "contentUrl" : test.get('value')[i].get('contentUrl'),
                "datePublished" : test.get('value')[i].get('datePublished'),
                "encodingFormat" : test.get('value')[i].get('encodingFormat'),
                "height" : test.get('value')[i].get('height'),
                "hostPageUrl" : test.get('value')[i].get('hostPageUrl'),
                "imageId" : test.get('value')[i].get('imageId'),
                "width" : test.get('value')[i].get('width')
                })
          with open(file_path,'w') as outfile:
            json.dump(data,outfile)
        except Exception as e: 
          print(e)
          error_log.append(f"{file_name}: {e}")
          pass
        
    # get back to base_path
    os.chdir("../..")
    
  # measure performance time
  time_taken = time.time()-start
  return time_taken, error_log

In [47]:
make_img_list(name_list, 3 ,key1)

Barak Obama_0
https://issuepedia.org/wikiup/thumb/f/f8/Officialportrait.jpg/1200px-Officialportrait.jpg
Barak Obama_1
https://images-na.ssl-images-amazon.com/images/I/B1iW0a3boMS.jpg
Barak Obama_2
https://i0.web.de/image/374/33558374,pd=1/barack-obama.jpg
Donald Trump_0
http://presencianoticias.com/wp-content/uploads/2018/05/trump-fail-004.jpg
HTTP Error 403: Forbidden
Donald Trump_1
https://celebrityinsider.org/wp-content/uploads/2020/05/d-trump.jpg
HTTP Error 403: Forbidden
Donald Trump_2
http://www.infoescola.com/wp-content/uploads/2016/05/donald-trump_308374256.jpg
HTTP Error 403: Forbidden
Joe Biden_0
https://arc-anglerfish-arc2-prod-bostonglobe.s3.amazonaws.com/public/FNYETSMRXNDK3IN6E2LVAHJAOQ.JPG
Joe Biden_1
http://tu18jax.com/wp-content/uploads/2020/11/Joe_Biden-scaled.jpg
Joe Biden_2
https://veja.abril.com.br/wp-content/uploads/2019/09/biden-eua-getty.jpg


(31.42984414100647,
 ['Donald Trump_0: HTTP Error 403: Forbidden',
  'Donald Trump_1: HTTP Error 403: Forbidden',
  'Donald Trump_2: HTTP Error 403: Forbidden'])